In [63]:
import pymysql.cursors
import datetime

connection = pymysql.connect(host='localhost',
                             user='tatelab1',
                             password='tatelab1',
                             db='names',
                             charset='utf8',
                             # 結果の受け取り方の指定。Dict形式で結果を受け取ることができる
                             cursorclass=pymysql.cursors.DictCursor)

In [72]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import colour
import PIL.ImageDraw
import sys

def heikin_img(path):

    source_file = path
    source = PIL.Image.open(source_file)

    color_arr = np.array(source)
    w_size, h_size, n_color = color_arr.shape
    color_arr = color_arr.reshape(w_size * h_size, n_color)

    color_mean = np.mean(color_arr, axis=0)
    color_mean = color_mean.astype(int)
    color_mean = tuple(color_mean)

    im = PIL.Image.new('RGB', (1, 1), color_mean)
    im.save('YYY.png')
    a = cv2.imread('YYY.png')
    return a
    
def heikin_hikaku(a,b):
    imgA = cv2.cvtColor(a, cv2.COLOR_BGR2Lab)
    imgB = cv2.cvtColor(b, cv2.COLOR_BGR2Lab)
    sys.getsizeof(imgA)
    return colour.delta_E(imgA, imgB)
    
def saido_img(path):
    img = cv2.imread(path)
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = img2[:,:,0], img2[:,:,1], img2[:,:,2]
    hist_h = cv2.calcHist([h],[0],None,[256],[0,256])
    return hist_h

def kidoyoko_img(path):
    img = cv2.imread(path)
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img2 = cv2.resize(img2,(400,400))
    img3 = np.array(img2)
    height,width,channels=img2.shape[:3]
    luminanceList=[]
    counter=0
    r=1*img3[:, :, 0] + 0*img3[:, :, 1] + 0*img3[:, :, 2]
    g=0*img3[:, :, 0] + 1*img3[:, :, 1] + 0*img3[:, :, 2]
    b=0*img3[:, :, 0] + 0*img3[:, :, 1] + 1*img3[:, :, 2]
    for i in range(width):
        sumR=np.mean(r, axis=0)
        sumG=np.mean(g, axis=0)
        sumB=np.mean(b, axis=0)
        luminance = ( 0.298912 * sumR[i] + 0.586611 * sumG[i] + 0.114478 * sumB[i] )

        luminanceList.append(luminance)

    luminanceList = np.round(luminanceList, 1) 
    return luminanceList

def kidotate_img(path):
    img = cv2.imread(path)
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img2 = cv2.resize(img2,(400,400))
    img3 = np.array(img2)
    height,width,channels=img2.shape[:3]
    luminanceList=[]
    counter=0
    r=1*img3[:, :, 0] + 0*img3[:, :, 1] + 0*img3[:, :, 2]
    g=0*img3[:, :, 0] + 1*img3[:, :, 1] + 0*img3[:, :, 2]
    b=0*img3[:, :, 0] + 0*img3[:, :, 1] + 1*img3[:, :, 2]
    for i in range(height):
        sumR=np.mean(r, axis=1)
        sumG=np.mean(g, axis=1)
        sumB=np.mean(b, axis=1)
        luminance = ( 0.298912 * sumR[i] + 0.586611 * sumG[i] + 0.114478 * sumB[i] )
        luminanceList.append(luminance)
    luminanceList = np.round(luminanceList, 1) 
    return luminanceList

In [75]:
import os
import glob
import time
import datetime

def Register_TodayCloth(file):
    heikin = heikin_img(file)
    saido = saido_img(file)
    kidoyoko = kidoyoko_img(file)
    kidotate = kidotate_img(file)
    name = os.path.splitext(os.path.basename(file))[0]
    print(name)
    
    with connection.cursor() as cursor:
        #sql ="UPDATE userdata2 set clothHeikin = %s, clothSaido = %s, clothTateKido = %s, clothYokoKido = %s, where name = %s"
        sql ="UPDATE userdata3 set date = %s, clothHeikin = %s, clothSaido = %s, clothTateKido = %s, clothYokoKido = %s where name = %s"
        cursor.execute(sql, ( str(datetime.date.today().strftime('%Y%m%d')),heikin, saido, kidotate, kidoyoko, name))

        connection.commit() #データをいじったあとはこれで確定する
    
path = '/home/ta1ty0/Search2021/Image'
Flag = False
while(True):
    list_of_files = sorted(glob.glob(path+'/Register/*'))
    if len(list_of_files) > 0:
        Flag = True
    else:
        Flag = False
    while(Flag):
        list_of_files = sorted(glob.glob(path+'/Register/*'))
        if (len(list_of_files)>1):
            latest_file = max(list_of_files, key=os.path.getctime)
            print(latest_file)
            Register_TodayCloth(latest_file)
            os.remove(latest_file)

        else:
            print(list_of_files[0])
            Register_TodayCloth(list_of_files[0])
            os.remove(list_of_files[0])
            Flag = False
    
    time.sleep(5)

/home/ta1ty0/Search2021/Image/Register/1_20210921_10:25.png
1_20210921_10:25
/home/ta1ty0/Search2021/Image/Register/hanako.png
hanako
/home/ta1ty0/Search2021/Image/Register/taro.png
taro
/home/ta1ty0/Search2021/Image/Register/yosio.png
yosio


KeyboardInterrupt: 